<a href="https://colab.research.google.com/github/BillWorstell/ISBI2020_TUTORIAL/blob/master/04_basic_registration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Basic Registration</h1>


**Summary:**

1. Creating an instance of the registration framework requires selection of the following components:
   * Optimizer.
   * Similarity metric.
   * Interpolator.
2. The registration framework only supports images with sitkFloat32 and sitkFloat64 pixel types (use the SimpleITK <a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#af8c9d7cc96a299a05890e9c3db911885">Cast()</a> function if your image's pixel type is something else).

3. Successful registration is highly dependent on initialization. In general you can:
   * Use auxiliary information or user interaction to obtain an initial transformation (avoid resampling).
   * Center the images using the <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1CenteredTransformInitializerFilter.html">CenteredTransformInitializer</a>.
   * Coarsely sample the parameter space using the <a href="https://itk.org/Doxygen/html/classitk_1_1ExhaustiveOptimizerv4.html">Exhaustive Optimizer</a> to obtain one or more initial transformation estimates.
   * Manually initialize, via direct manipulation of transformation parameters and visualization or localization of corresponding points in the two images and then use the <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1LandmarkBasedTransformInitializerFilter.html">LandmarkBasedTransformInitializer</a>.

## Install SimpleITK and Environment

Expects to find on MyDrive:
 
/gdrive/MyDrive/SimpleITK-Notebooks/Python

/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt

/gdrive/MyDrive/Fiji.app/ImageJ-linux64

<img src="figures/ImageOriginAndSpacing.png" style="width:700px"/><br><br>

In [1]:
pip install virtualenv

     |████████████████████████████████| 7.2MB 7.4MB/s 
     |████████████████████████████████| 337kB 50.4MB/s 


In [2]:
#!virtualenv ~/sitkpy --no-site-packages
!virtualenv ~/sitkpy

created virtual environment CPython3.7.10.final.0-64 in 990ms
  creator CPython3Posix(dest=/root/sitkpy, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.0.1, setuptools==52.0.0, wheel==0.36.2
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
#!pip install -r '/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt'
!~/sitkpy/bin/pip install -r '/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt'

     |████████████████████████████████| 47.4 MB 42 kB/s 
     |████████████████████████████████| 11.5 MB 47.8 MB/s 
     |████████████████████████████████| 121 kB 64.3 MB/s 
     |████████████████████████████████| 15.3 MB 159 kB/s 
     |████████████████████████████████| 27.4 MB 66 kB/s 
     |████████████████████████████████| 9.9 MB 46.6 MB/s 
     |████████████████████████████████| 3.2 MB 67.8 MB/s 
     |████████████████████████████████| 108 kB 59.0 MB/s 
     |████████████████████████████████| 243 kB 60.4 MB/s 
     |████████████████████████████████| 120 kB 39.6 MB/s 
     |████████████████████████████████| 784 kB 59.0 MB/s 
     |████████████████████████████████| 100 kB 9.3 MB/s 
     |████████████████████████████████| 2.2 MB 63.4 MB/s 
     |████████████████████████████████| 113 kB 36.1 MB/s 
     |████████████████████████████████| 428 kB 61.1 MB/s 
     |████████████████████████████████| 108 kB 60.9 MB/s 
     |████████████████████████████████| 1.4 MB 47.9 MB/s 
     |██████████

In [5]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [6]:
pip install SimpleITK

  Using cached https://files.pythonhosted.org/packages/9c/6b/85df5eb3a8059b23a53a9f224476e75473f9bcc0a8583ed1a9c34619f372/SimpleITK-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl


In [7]:
pip install ipywidgets

In [8]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [9]:
%ls -ltr '/gdrive/MyDrive/Fiji.app/ImageJ-linux64'

-rw------- 1 root root 90243 Feb 27 18:15 /gdrive/MyDrive/Fiji.app/ImageJ-linux64


In [10]:
!pip install itk

#!~/sitkpy/bin/pip install ITK

     |████████████████████████████████| 10.3MB 5.2MB/s 
     |████████████████████████████████| 50.1MB 88kB/s 
     |████████████████████████████████| 68.0MB 52kB/s 
     |████████████████████████████████| 14.4MB 353kB/s 
     |████████████████████████████████| 14.0MB 326kB/s 
     |████████████████████████████████| 36.3MB 126kB/s 


In [11]:
!echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [12]:
import os
#os.environ['PYTHONPATH'] += ":/content/gdrive/My Drive/Colab Notebooks/MNIST_Classifier/src"
os.environ['PATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/content/gdrive/My Drive/Fiji.app


In [13]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [14]:
import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data, fetch_data_all

from ipywidgets import interact

print(sitk.Version())

SimpleITK Version: 2.0.2 (ITK 5.1)
Compiled: Dec  1 2020 22:01:03



In [15]:
cd /usr/local/

/usr/local


In [16]:
import shutil

In [17]:
%ls /gdrive/MyDrive/

'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [18]:
shutil.copyfile('/gdrive/MyDrive/fiji-linux64-20170530.zip', '/usr/local/fiji-linux64-20170530.zip')

'/usr/local/fiji-linux64-20170530.zip'

In [19]:
cd /usr/local

/usr/local


In [20]:
!unzip ./fiji-linux64-20170530.zip

Archive:  ./fiji-linux64-20170530.zip
  inflating: Fiji.app/db.xml.gz      
  inflating: Fiji.app/Contents/Info.plist  
  inflating: Fiji.app/Contents/Resources/Fiji.icns  
  inflating: Fiji.app/Contents/Resources/ImageJ.icns  
  inflating: Fiji.app/ImageJ-linux64  
  inflating: Fiji.app/plugins/3D_Blob_Segmentation-3.0.0.jar  
  inflating: Fiji.app/plugins/3D_Objects_Counter-2.0.0.jar  
  inflating: Fiji.app/plugins/3D_Viewer-4.0.1.jar  
  inflating: Fiji.app/plugins/Algorithm_Launcher.jar  
  inflating: Fiji.app/plugins/Analyze/Dynamic_ROI_Profiler.clj  
  inflating: Fiji.app/plugins/Analyze/Measure_RGB.txt  
  inflating: Fiji.app/plugins/AnalyzeSkeleton_-3.1.2.jar  
  inflating: Fiji.app/plugins/Anisotropic_Diffusion_2D-2.0.0.jar  
  inflating: Fiji.app/plugins/Archipelago_Plugins-0.5.2.jar  
  inflating: Fiji.app/plugins/Arrow_-2.0.1.jar  
  inflating: Fiji.app/plugins/Auto_Threshold-1.16.4.jar  
  inflating: Fiji.app/plugins/BalloonSegmentation_-3.0.0.jar  
  inflating: Fiji.app/p

In [21]:
#!pip install SimpleITK

In [22]:
#!git clone https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks.git

In [23]:
pip install ipywidgets

In [24]:
# this will allow the notebook to reload/refresh automatically within the runtime
%reload_ext autoreload
%autoreload 2

from ipywidgets import interact

def f(x):
  return x

interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f>

In [25]:
#pip install itk

In [26]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python/'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [27]:
!python '/gdrive/MyDrive/SimpleITK-Notebooks/Python/update_path_to_download_script.py'

In [28]:
import os
os.environ['PYTHONPATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PYTHONPATH
os.environ['PATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PATH

/env/python:/content/gdrive/My Drive/Fiji.app
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/content/gdrive/My Drive/Fiji.app:/content/gdrive/My Drive/Fiji.app


In [29]:
import numpy as np
import os
from ipywidgets import interact, fixed

%matplotlib inline
import matplotlib.pyplot as plt

from downloaddata import fetch_data as fdata

OUTPUT_DIR = 'output'

image_viewer = sitk.ImageViewer()

In [30]:
print(image_viewer)

itk::simple::ImageViewer
  Title: 
  Command: %a -eval 'open("%f"); rename("%t");'
  Application: /usr/local/Fiji.app/ImageJ-linux64
  Default Application: /usr/local/Fiji.app/ImageJ-linux64
  File Extension: .mha
  Default File Extension: .mha
  Search Path: [ ./, /root/bin/, /opt/, /usr/local/ ]
  Executable Names: [ Fiji.app/ImageJ-linux64, Fiji.app/ImageJ-linux32 ]
  Debug Flag: 0



In [31]:
%ls /usr/local/Fiji.app/ImageJ-linux64

/usr/local/Fiji.app/ImageJ-linux64*


In [32]:
#!python '/gdrive/MyDrive/SimpleITK-Notebooks/Python/update_path_to_download_script.py'

In [33]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python/'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [34]:
import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data, fetch_data_all

from ipywidgets import interact

print(sitk.Version())

SimpleITK Version: 2.0.2 (ITK 5.1)
Compiled: Dec  1 2020 22:01:03



In [35]:
%cd '/gdrive/MyDrive/'
%ls

/gdrive/MyDrive
'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [36]:
cd /usr/local/

/usr/local


In [37]:
#image_viewer.SetApplication('/usr/local/Fiji.app/ImageJ-linux64')

In [38]:
# Uncomment the line below to change the default external viewer to your viewer of choice and test that it works.
#%env SITK_SHOW_COMMAND /Applications/ITK-SNAP.app/Contents/MacOS/ITK-SNAP 
#%env SITK_SHOW_COMMAND  /gdrive/MyDrive/Fiji.app/ImageJ-linux64
#%env SITK_SHOW_COMMAND  /usr/local/Fiji.app/
#%env SITK_SHOW_COMMAND  /usr/local/Fiji.app/ImageJ-linux64

# Retrieve an image from the network, read it and display using the external viewer. 
# The show method will also set the display window's title and by setting debugOn to True, 
# will also print information with respect to the command it is attempting to invoke.
# NOTE: The debug information is printed to the terminal from which you launched the notebook
#       server.

In [39]:
%ls /gdrive/MyDrive/

'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [40]:
%cd /gdrive/MyDrive/SimpleITK_ISBI2020_TUTORIAL

/gdrive/MyDrive/SimpleITK_ISBI2020_TUTORIAL


In [41]:
print(image_viewer)

itk::simple::ImageViewer
  Title: 
  Command: %a -eval 'open("%f"); rename("%t");'
  Application: /usr/local/Fiji.app/ImageJ-linux64
  Default Application: /usr/local/Fiji.app/ImageJ-linux64
  File Extension: .mha
  Default File Extension: .mha
  Search Path: [ ./, /root/bin/, /opt/, /usr/local/ ]
  Executable Names: [ Fiji.app/ImageJ-linux64, Fiji.app/ImageJ-linux32 ]
  Debug Flag: 0



In [42]:
#import SimpleITK as sitk

#%matplotlib inline
#import matplotlib.pyplot as plt
#import numpy as np

#from ipywidgets import interact, fixed
#import os

#OUTPUT_DIR = 'Output'

# Utility method that either downloads data from the Girder repository or
# if already downloaded returns the file name for reading from disk (cached data).
#%run update_path_to_download_script
#from downloaddata import fetch_data as fdata

## Registration Components 

<img src="figures/ITKv4RegistrationComponentsDiagram.svg" style="width:700px"/><br><br>

There are many options for creating an instance of the registration framework, all of which are configured in SimpleITK via methods of the <a href="http://www.itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageRegistrationMethod.html">ImageRegistrationMethod</a> class. This class encapsulates many of the components available in ITK for constructing a registration instance.

Currently, the available choices from the following groups of ITK components are:

### Optimizers

The SimpleITK registration framework supports several optimizer types via the SetOptimizerAsX() methods, these include:

<ul>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1ExhaustiveOptimizerv4.html">Exhaustive</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1AmoebaOptimizerv4.html">Nelder-Mead downhill simplex</a>, a.k.a. Amoeba.
  </li>
  <li>
   <a href="https://itk.org/Doxygen/html/classitk_1_1PowellOptimizerv4.html">Powell optimizer</a>.
  </li>
  <li>
   <a href="https://itk.org/Doxygen/html/classitk_1_1OnePlusOneEvolutionaryOptimizerv4.html">1+1 evolutionary optimizer</a>.
  </li>
  <li>
  Variations on gradient descent:
  <ul>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1GradientDescentOptimizerv4Template.html">GradientDescent</a>
    </li>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1GradientDescentLineSearchOptimizerv4Template.html">GradientDescentLineSearch</a>
    </li>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1RegularStepGradientDescentOptimizerv4.html">RegularStepGradientDescent</a>
    </li>
  </ul>
  </li>
  <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1ConjugateGradientLineSearchOptimizerv4Template.html">ConjugateGradientLineSearch</a> 
  </li>
  <li>
  <a href="https://itk.org/Doxygen/html/classitk_1_1LBFGS2Optimizerv4.html">L-BFGS2</a> (Limited memory Broyden,  Fletcher,Goldfarb,Shannon)
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html">L-BFGS-B</a> (Limited memory Broyden,  Fletcher,Goldfarb,Shannon-Bound Constrained) - supports the use of simple constraints ($l\leq x \leq u$)  
  </li>
    
</ul>

 
### Similarity metrics

The SimpleITK registration framework supports several metric types via the SetMetricAsX() methods, these include:

<ul>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1MeanSquaresImageToImageMetricv4.html">MeanSquares</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1DemonsImageToImageMetricv4.html">Demons</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1CorrelationImageToImageMetricv4.html">Correlation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1ANTSNeighborhoodCorrelationImageToImageMetricv4.html">ANTSNeighborhoodCorrelation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1JointHistogramMutualInformationImageToImageMetricv4.html">JointHistogramMutualInformation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1MattesMutualInformationImageToImageMetricv4.html">MattesMutualInformation</a>
  </li>
</ul>


### Interpolators

The SimpleITK registration framework supports several interpolators via the SetInterpolator() method, which receives one of
the <a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#a7cb1ef8bd02c669c02ea2f9f5aa374e5">following enumerations</a>:
<ul>
<li> sitkNearestNeighbor </li>
<li> sitkLinear </li>
<li> sitkBSpline </li>
<li> sitkGaussian </li>
<li> sitkHammingWindowedSinc </li>
<li> sitkCosineWindowedSinc </li>
<li> sitkWelchWindowedSinc </li>
<li> sitkLanczosWindowedSinc </li>
<li> sitkBlackmanWindowedSinc </li>
</ul>

In [43]:
import SimpleITK as sitk
from downloaddata import fetch_data as fdata
%matplotlib notebook
import gui
import registration_gui as rgui

import numpy as np
import os
OUTPUT_DIR = 'output'

## Read images

We first read the images, specifying the pixel type that is required for registration (Float32 or Float64) and look at them. In this notebook we use a CT and MR image from the same patient. These are part of the training data from the Retrospective Image Registration Evaluation (<a href="http://www.insight-journal.org/rire/">RIRE</a>) project.

In [44]:
fixed_image =  sitk.ReadImage(fdata("training_001_ct.mha"), sitk.sitkFloat32)
moving_image = sitk.ReadImage(fdata("training_001_mr_T1.mha"), sitk.sitkFloat32)

ct_window_level = [835,162]
mr_window_level = [1036,520]

gui.MultiImageDisplay(image_list = [fixed_image, moving_image],                   
                      title_list = ['fixed', 'moving'], figure_size=(8,4), window_level_list=[ct_window_level, mr_window_level]);

Fetching training_001_ct.mha
Fetching training_001_mr_T1.mha


<IPython.core.display.Javascript object>

## Classic Registration

Estimate a 3D rigid transformation between images of different modalities. 

We have made the following choices with respect to initialization and registration component settings:

<ul>

<li>Similarity metric, mutual information (Mattes MI):
<ul>
  <li>Number of histogram bins, 50.</li>
  <li>Sampling strategy, random.</li>
  <li>Sampling percentage, 1%.</li>
</ul>
</li>
<li>Interpolator, sitkLinear.</li>
<li>Optimizer, gradient descent: 
<ul>
  <li>Learning rate, step size along traversal direction in parameter space, 1.0 .</li>
  <li>Number of iterations, maximal number of iterations, 100.</li>
  <li>Convergence minimum value, value used for convergence checking in conjunction with the energy profile of the similarity metric that is estimated in the given window size, 1e-6.</li>
  <li>Convergence window size, number of values of the similarity metric which are used to estimate the energy profile of the similarity metric, 10.</li>
</ul>
</li>
</ul>

We initialize registration by aligning the centers of the two volumes. To qualitatively evaluate the result we use a linked cursor approach, click on one image and the corresponding point is added to the other image.

In [45]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=initial_transform, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

Run the next cell three times:
1. As is.
2. Uncomment the automated optimizer scale setting so that a change in rotation (radians) has a similar effect to a change in translation (mm).
3. Uncomment the multi-resolution settings.

In [46]:
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings.
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
#registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework.            
#registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
#registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
#registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Don't optimize in-place, we would possibly like to run this cell multiple times.
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

final_transform = registration_method.Execute(fixed_image, moving_image)

# Always check the reason optimization terminated.
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

<IPython.core.display.Javascript object>

Final metric value: -0.42536448843127295
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 14.


Qualitatively evaluate the result using a linked cursor approach (visual evaluation):

In [47]:
gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=final_transform,fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

If we are satisfied with the results, save them to file.

In [48]:
moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
sitk.WriteImage(moving_resampled, os.path.join(OUTPUT_DIR, 'RIRE_training_001_mr_T1_resampled.mha'))
sitk.WriteTransform(final_transform, os.path.join(OUTPUT_DIR, 'RIRE_training_001_CT_2_mr_T1.tfm'))

## ITKv4 Coordinate Systems

Unlike the classical registration approach where the fixed and moving images are treated differently, the ITKv4 registration framework allows you to treat both images in the same manner. This is achieved by introducing a third coordinate system, the virtual image domain.

<img src="figures/registrationFrameworkTransformations.svg"/><br><br>

Thus, the ITK v4 registration framework deals with three transformations:
<ul>
<li>
SetInitialTransform, $T_{opt}$ - composed with the moving initial transform, maps points from the virtual image domain to the moving image domain, modified during optimization. 
</li>
<li>
SetFixedInitialTransform $T_f$- maps points from the virtual image domain to the fixed image domain, never modified.
</li>
<li>
SetMovingInitialTransform $T_m$- maps points from the virtual image domain to the moving image domain, never modified.
</li>
</ul>

The transformation that maps points from the fixed to moving image domains is thus: $^M\mathbf{p}  = T_{opt}(T_m(T_f^{-1}(^F\mathbf{p})))$

We now modify the previous example to use $T_{opt}$ and $T_m$.

In [49]:
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings.
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Setup for the multi-resolution framework.            
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Set the initial moving and optimized transforms.
optimized_transform = sitk.Euler3DTransform()    
registration_method.SetMovingInitialTransform(initial_transform)
registration_method.SetInitialTransform(optimized_transform, inPlace=False)

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

# Need to compose the transformations after registration.
final_transform_v4 = registration_method.Execute(fixed_image, moving_image)
#final_transform_v4.AddTransform(initial_transform)

# Always check the reason optimization terminated.
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

<IPython.core.display.Javascript object>

Final metric value: -0.5688043714490768
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 9.


In [50]:
gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=final_transform_v4, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

## Initialization

Initialization effects both the runtime and convergence to the correct minimum. Ideally our transformation is initialized close to the correct solution ensuring convergence in a timely manner. Problem specific initialization will often yield better results than the more generic solutions we show below. As a rule of thumb, use as much prior information (external to the image content) as you can to initialize your registration task.

Common initializations in the generic setting:
1. Do nothing (a.k.a. hope/unique setting) - initialize using the identity transformation.
2. CenteredTransformInitializer (GEOMETRY or MOMENTS) - translation based initialization, align the centers of the images or their centers of mass (intensity based).
3. Use the exhaustive optimizer as a first step - never underestimate brute force.
4. Manual initialization - allow an operator to control parameter settings using a GUI with visual feedback or identify multiple corresponding points in the two images. 


We start by loading our data, CT and MR scans of the CIRS (Norfolk, VA, USA) abdominal phantom.

In [51]:
data_directory = os.path.dirname(fdata("CIRS057A_MR_CT_DICOM/readme.txt"))

ct_window_level = [1727,-320]
mr_window_level = [355,178]

fixed_series_ID = "1.2.840.113619.2.290.3.3233817346.783.1399004564.515"
moving_series_ID = "1.3.12.2.1107.5.2.18.41548.30000014030519285935000000933"

fixed_series_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, fixed_series_ID)
moving_series_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, moving_series_ID)

fixed_image = sitk.ReadImage(fixed_series_filenames, sitk.sitkFloat32)
moving_image = sitk.ReadImage(moving_series_filenames, sitk.sitkFloat32)

Fetching CIRS057A_MR_CT_DICOM/readme.txt


### Identity transform initialization

In [52]:
initial_transform = sitk.Transform()
gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=initial_transform, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

When working with clinical images, the DICOM tags define the orientation and position of the anatomy in the volume. The tags of interest are:
<ul>
  <li> (0020|0032) Image Position (Patient) : coordinates of the the first transmitted voxel. </li>
  <li>(0020|0037) Image Orientation (Patient): directions of first row and column in 3D space. </li>
  <li>(0018|5100) Patient Position: Patient placement on the table 
  <ul>
  <li> Head First Prone (HFP)</li>
  <li> Head First Supine (HFS)</li>
  <li> Head First Decubitus Right (HFDR)</li>
  <li> Head First Decubitus Left (HFDL)</li>
  <li> Feet First Prone (FFP)</li>
  <li> Feet First Supine (FFS)</li>
  <li> Feet First Decubitus Right (FFDR)</li>
  <li> Feet First Decubitus Left (FFDL)</li>
  </ul>
  </li>
</ul>

SimpleITK/ITK takes this information into account when loading DICOM images. 

But we are working with DICOM images, so why aren't the images oriented correctly using the identity transformation?

Well, the patient position in the scanner is manually entered by the technician meaning that errors may occur, though rarely. For our data, a phantom, it is unclear which side is the "head" and which is the "feet" so the technicians entered reasonable values for each scan. 

In [53]:
reader = sitk.ImageFileReader()
reader.SetFileName(fixed_series_filenames[0])
reader.ReadImageInformation()
print('Patient name: ' + reader.GetMetaData('0010|0010') + ', Patient position:' + reader.GetMetaData('0018|5100'))
reader.SetFileName(moving_series_filenames[0])
reader.ReadImageInformation()
print('Patient name: ' + reader.GetMetaData('0010|0010') + ', Patient position:' + reader.GetMetaData('0018|5100'))

Patient name: test, Patient position:HFS 
Patient name: ZY05MAR2014 , Patient position:HFS 


### CenteredTransformInitializer initialization 
Compare GEOMETRY and MOMENTS based approaches:

In [54]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=initial_transform, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

### Exhaustive optimizer initialization

The following initialization approach is a combination of using prior knowledge and the exhaustive optimizer. We know that the scans are acquired with the "patient" either supine (on their back) or prone (on their stomach) and that the scan direction (head-to-feet or feet-to-head) is along the images' z axis. 
We use the CenteredTransformInitializer to initialize the translation and the exhaustive optimizer to obtain an initial rigid transformation.

The exhaustive optimizer evaluates the similarity metric on a grid in parameter space centered on the parameters of the initial transform. This grid is defined using three elements:
1. numberOfSteps.
2. stepLength.
3. optimizer scales.

The similarity metric is evaluated on the resulting parameter grid:
initial_parameters &plusmn; numberOfSteps &times; stepLength &times; optimizerScales

***Example***:
1. numberOfSteps=[1,0,2,0,0,0]
2. stepLength = np.pi
3. optimizer scales = [1,1,0.5,1,1,1]

Will perform 15 metric evaluations ($\displaystyle\prod_i (2*numberOfSteps[i] + 1)$).

The parameter values for the second parameter and the last three parameters are the initial parameter values. The parameter values for the first parameter are $v_{init}-\pi, v_{init}, v_{init}+\pi$ and the parameter values for the third parameter are $v_{init}-\pi, v_{init}-\pi/2, v_{init}, v_{init}+\pi/2, v_{init}+\pi$.

The transformation corresponding to the lowest similarity metric is returned.

In [55]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
# The order of parameters for the Euler3DTransform is [angle_x, angle_y, angle_z, t_x, t_y, t_z]. The parameter 
# sampling grid is centered on the initial_transform parameter values, that are all zero for the rotations. Given
# the number of steps, their length and optimizer scales we have:
# angle_x = 0
# angle_y = -pi, 0, pi
# angle_z = -pi, 0, pi
registration_method.SetOptimizerAsExhaustive(numberOfSteps=[0,1,1,0,0,0], stepLength = np.pi)
registration_method.SetOptimizerScales([1,1,1,1,1,1])

#Perform the registration in-place so that the initial_transform is modified.
registration_method.SetInitialTransform(initial_transform, inPlace=True)
registration_method.Execute(fixed_image, moving_image)

gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=initial_transform, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

#### Manual initialization

When all else fails, a human in the loop will almost always be able to robustly initialize the registration.

In the example below we identify corresponding points to compute an initial rigid transformation. 

**Note**: There is no correspondence between the fiducial markers on the phantom.

In [56]:
point_acquisition_interface = gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

In [57]:
# Get the manually specified points and compute the transformation.

fixed_image_points, moving_image_points = point_acquisition_interface.get_points()

# Previously localized points (here so that the testing passes):
fixed_image_points = [(24.062587103074605, 14.594981536981521, -58.75), (6.178716135332678, 53.93949766601378, -58.75), (74.14383149714774, -69.04462737237648, -76.25), (109.74899278747029, -14.905272533666817, -76.25)]
moving_image_points = [(4.358707846364581, 60.46357110706131, -71.53120422363281), (24.09010295252645, 98.21840981673873, -71.53120422363281), (-52.11888008581127, -26.57984635768439, -58.53120422363281), (-87.46150681392184, 28.73904765153219, -58.53120422363281)]

fixed_image_points_flat = [c for p in fixed_image_points for c in p]        
moving_image_points_flat = [c for p in moving_image_points for c in p]
initial_transformation = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), 
                                                                fixed_image_points_flat, 
                                                                moving_image_points_flat)
gui.RegistrationPointDataAquisition(fixed_image, moving_image, figure_size=(8,4), known_transformation=initial_transform, fixed_window_level=ct_window_level, moving_window_level=mr_window_level);

<IPython.core.display.Javascript object>

<a href="05_advanced_registration.ipynb"><h2 align=right>Next &raquo;</h2></a>